In [11]:
import re
import emoji
import string
import unicodedata
import nltk
from underthesea import word_tokenize, pos_tag
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
from collections import defaultdict
from vncorenlp import VnCoreNLP


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/quytien/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/quytien/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/quytien/nltk_data...


True

### 1. Loại bỏ ký tự đặc biệt, dấu câu, emoji 
#### *- Loại bỏ ký tự đặc biệt không xoá các ký tự tiếng Việt (Unicode property): **r"[^\w\sÀ-ỹ]"*** <br /> *- Loại bỏ toàn bộ ký tự đặc biệt, chỉ giữ lại các ký tự ASCII (tiếng Anh không dấu): **r"[^a-zA-Z0-9]"***

In [7]:
txt = "Chỉ cầ$n kiên nh@ẫn rồi mọi t.hứ sẽ đến."

new_string = re.sub(r"[^\w\sÀ-ỹ]","",txt)
print(new_string)

Chỉ cần kiên nhẫn rồi mọi thứ sẽ đến


#### Loại bỏ dấu câu

In [15]:
text = "Cuộc, sống không? phải lúc nào; cũng như mình: muốn!"

new_string = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
print(new_string)

Cuộc sống không phải lúc nào cũng như mình muốn


#### Loại bỏ emoji

In [20]:
text = "Hôm nay trời đẹp quá😍🌞! Đi chơi thôi 🏖️🚗"

new_string = emoji.replace_emoji(text, replace='')
print(new_string)

Hôm nay trời đẹp quá! Đi chơi thôi 


### 2.	Chuẩn hóa và tách từ tiếng Việt
#### Tách từ sử dụng thư viện underthesea (hỗ trợ version 10, 11)

In [2]:
txt = "Và em luôn tin sau cơn mưa cầu vồng sẽ lấp lánh."

tokens = word_tokenize(txt, format="text")
print(tokens)

Và em luôn tin sau cơn mưa cầu vồng sẽ lấp_lánh .


#### Tách từ sử dụng thư viện VnCoreNLP

In [8]:
vncorenlp = VnCoreNLP(
    "/Users/quytien/VnCoreNLP/VnCoreNLP-1.2.jar", 
    annotators="wseg", 
    max_heap_size='-Xmx2g'
)
res = vncorenlp.tokenize(txt)
print(res)

[['Và', 'em', 'luôn', 'tin', 'sau', 'cơn', 'mưa', 'cầu_vồng', 'sẽ', 'lấp_lánh', '.']]


#### Chuẩn hoá

In [8]:
txt_1 = "Và               em luôn tin sau cơn mưa cầu vồng sẽ lấp lánh."
# Chuẩn hóa Unicode
txt_1 = unicodedata.normalize('NFC', txt_1)

# Chuyển về chữ thường
txt_1 = txt_1.lower()

# Loại bỏ khoảng trắng dư thừa
txt_1 = re.sub(r"\s+", " ", txt_1).strip()
print(txt_1)

và em luôn tin sau cơn mưa cầu vồng sẽ lấp lánh.


### 3.	Loại bỏ stopwords và stemming/lemmatization
#### Loại bỏ stopwords

Thư viện nltk chưa hỗ trợ tiếng Việt

In [15]:
english_stopwords = stopwords.words('english')
print(english_stopwords)

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [12]:
def load_vietnamese_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return set(line.strip().lower() for line in f if line.strip())

vietnamese_stopwords = load_vietnamese_stopwords("vietnamese-stopwords.txt")

In [18]:
text_stopwords = "Ta từng bắt gặp nhau ở khắp Sài Gòn chắc là lúc còn yêu dù muốn tránh cũng khó"
# Tách từ
tokens_stopwords = word_tokenize(text_stopwords, format="text").split()
# Lọc stopwords
filtered_tokens = [word for word in tokens_stopwords if word.lower() not in vietnamese_stopwords]
print(filtered_tokens)

['Ta', 'bắt_gặp', 'khắp', 'Sài_Gòn', 'yêu']


#### Stemming/ Lemmatization

#### Stemming

In [8]:
stemmer = PorterStemmer()
input_str = "There are several types of stemming algorithms."
input_str = nltk.word_tokenize(input_str)
for word in input_str:
    print(stemmer.stem(word))

there
are
sever
type
of
stem
algorithm
.


#### Lemmatization

In [12]:
lemmatizer = WordNetLemmatizer()
input_str = "been had done languages cities mice"
input_str = nltk.word_tokenize(input_str)

for word in input_str:
    print(lemmatizer.lemmatize(word))

been
had
done
language
city
mouse


### 4.	Xây dựng bộ từ điển n-gram từ văn bản

In [12]:
def build_ngram(text, n=1):
    text
    tokens = word_tokenize(text.lower(), format="text").split()
    ngrams = defaultdict(int)
    for i in range (len(tokens) - n + 1):
        ngram = ' '.join(tokens[i:i+n])
        ngrams[ngram] += 1
    return dict(ngrams)

text = "Go, go, go, go! Don’t stop!"
bigram_dict = build_ngram(text, 1)

print(bigram_dict)

{'go': 4, ',': 3, '!': 2, 'don’t': 1, 'stop': 1}
